In [1]:
# serve python 3.11.9
import os 
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('SVD Big') \
    .master('spark://141.250.44.48:7077') \
    .config('spark.driver.memory', '2g') \
    .config('spark.executor.instances', '2') \
    .config('spark.executor.memory', '2g') \
    .config("spark.locality.wait.node", "0") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:50054") \
    .config("spark.sql.pivotMaxValues", "99999") \
    .getOrCreate()
    # pivot max values 
    # .config('spark.executor.instances', '2') \

In [3]:
import pandas as pd
import numpy as np

ratings = spark.read.csv('hdfs://localhost:50054/test/ratings.csv', inferSchema=True, header=True)
#ratings = spark.read.csv('hdfs://localhost:50054/test/ratingsBig.csv', inferSchema=True, header=True)
ratings.limit(5).toPandas()

Py4JJavaError: An error occurred while calling o37.read.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:92)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:75)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.DataFrameRea

In [4]:
# movies = spark.read.csv('hdfs://localhost:50054/test/movies.csv', inferSchema=True, header=True)
movies = spark.read.csv('hdfs://localhost:50054/test/moviesBig.csv', inferSchema=True, header=True)
movies.limit(5).toPandas()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
user_movie_ratings = ratings.join(movies, on='movieId', how='inner')
user_movie_ratings.limit(5).toPandas()

,movieId,userId,rating,timestamp,title,genres
0,1,1,4.0,1225734739,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,110,1,4.0,1225865086,Braveheart (1995),Action|Drama|War
2,158,1,4.0,1225733503,Casper (1995),Adventure|Children
3,260,1,4.5,1225735204,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
4,356,1,5.0,1225735119,Forrest Gump (1994),Comedy|Drama|Romance|War


In [ ]:
# convert the ratings to a mllib.linalg.SparseVector

from scipy.sparse import csr_matrix

def to_sparse_vector(ratings):
    num_users = ratings.select('userId').distinct().count()
    num_movies = ratings.select('movieId').distinct().count()
    ratings = ratings.rdd.map(lambda r: (r.userId, r.movieId, r.rating))
    rows, cols, data = zip(*ratings.collect())
    return csr_matrix((data, (rows, cols)), shape=(num_users, num_movies))

ratings = to_sparse_vector(ratings)

In [6]:
# user_movie_ratings_pivot = user_movie_ratings.groupBy("userId").pivot("movieId").agg(max("rating")).fillna(0)
user_movie_ratings_pivot = user_movie_ratings.groupBy("userId").pivot("movieId").max("rating").fillna(0) # <- Java Heap Space
user_movie_ratings_pivot.limit(5).toPandas()

Py4JJavaError: An error occurred while calling o75.na.
: java.lang.OutOfMemoryError: Java heap space


In [7]:
print(user_movie_ratings_pivot.count(), len(user_movie_ratings_pivot.columns)) 
print(type(user_movie_ratings_pivot))
user_movie_ratings_pivot.limit(5).toPandas()
# user_movie_ratings_pivot.printSchema()

610 9725
<class 'pyspark.sql.dataframe.DataFrame'>


,userId,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,471,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Optimizing the data
# ratings = 1
# # From the ALS documentation, genres and timestamps do not have an impact on the recommendations
# user_movie_ratings = user_movie_ratings.drop('genres')
# user_movie_ratings = user_movie_ratings.drop('timestamp')
# user_movie_ratings = user_movie_ratings.repartition(20).cache()

In [9]:
# Conversione in formato vettoriale per SVD

from pyspark.ml.linalg import Vectors
from pyspark.sql import Row

# Conversione in formato vettoriale per SVD
def row_to_vector(row):
    return Row(userId=row[0], featureVector=Vectors.dense(row[1:]))

user_movie_rdd = user_movie_ratings_pivot.rdd.map(lambda row: row_to_vector(row))
user_movie_df = spark.createDataFrame(user_movie_rdd)

In [10]:
from pyspark.ml.feature import StandardScaler

# Standardizzazione dei dati prima di applicare SVD
scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatures", withMean=True, withStd=True)
scaler_model = scaler.fit(user_movie_df)
scaled_data = scaler_model.transform(user_movie_df)

In [ ]:
# 

In [11]:
from scipy.sparse.linalg import svds
# Applicazione di SVD
# Conversione dei dati in matrice numpy per SVD
feature_matrix = np.array(scaled_data.select("scaledFeatures").rdd.map(lambda row: row[0].toArray()).collect())
U, sigma, Vt = svds(feature_matrix, k=20)  # k: numero di feature latenti

In [12]:
# Ricostruzione approssimata della matrice utente-film
sigma_diag = np.diag(sigma)
approx_user_movie_matrix = np.dot(np.dot(U, sigma_diag), Vt)

In [ ]:
approx_user_movie_matrix.show(5)

In [ ]:
#Test the predictions

user = 10

# take the user row from the 

In [ ]:
# creazione dataframe dove mantengo per ogni utente i film non visti e il rating previsto

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType, FloatType

def get_unseen_movies(ratings_list):
    return [i for i, e in enumerate(ratings_list) if e == 0]

def get_predicted_ratings(ratings_list):
    return [e for e in ratings_list]

get_unseen_movies_udf = udf(get_unseen_movies, ArrayType(IntegerType()))
get_predicted_ratings_udf = udf(get_predicted_ratings, ArrayType(FloatType()))

# ordino i film per rating previsto

from pyspark.sql.functions import col

def sort_unseen_movies(unseen_movies, predicted_ratings):
    unseen_movies = [x for _, x in sorted(zip(predicted_ratings, unseen_movies), key=lambda pair: pair[0], reverse=True)]
    return unseen_movies

sort_unseen_movies_udf = udf(sort_unseen_movies, ArrayType(IntegerType()))

# restitusco i primi 10 film per utente

def get_top_n_movies(unseen_movies, n):
    return unseen_movies[:n]

get_top_n_movies_udf = udf(get_top_n_movies, ArrayType(IntegerType()))



In [13]:
# Creazione DataFrame predetto
user_movie_predictions = spark.createDataFrame(
    [(int(user_id), Vectors.dense(predictions)) for user_id, predictions in enumerate(approx_user_movie_matrix)],
    ["userId", "predictions"]
)

In [14]:
user_movie_predictions.limit(5).toPandas()

,userId,predictions
0,0,"[-0.1685322140049568, -0.17813358052145772, -0..."
1,1,"[-0.17370934272320113, -0.1449819649053615, -0..."
2,2,"[-0.19931914166161815, -0.20114371308060933, -..."
3,3,"[-0.1953218666636986, -0.191697474250464, -0.1..."
4,4,"[-0.14947918540401733, -0.1611975042639148, -0..."


In [ ]:
# per ogni utente, usare cosine similarity per trovare i film più simili
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, expr

# Normalizzazione dei vettori di predizione

normalizer = Normalizer(inputCol="predictions", outputCol="normPredictions")
user_movie_predictions = normalizer.transform(user_movie_predictions)

# Calcolo della similarità coseno tra i vettori di predizione
def dot_product(v1, v2):
    return float(np.dot(v1, v2))

dot_product_udf = expr("dot(normPredictions, prediction)")

user_movie_predictions = user_movie_predictions.crossJoin(user_movie_predictions.select(col("userId").alias("userId2"), col("normPredictions").alias("prediction2")))
user_movie_predictions = user_movie_predictions.withColumn("dotProduct", dot_product_udf)


# Filtraggio dei film già visti

user_movie_ratings_pivot = user_movie_ratings_pivot.withColumn("userId", col("userId").cast("int"))
user_movie_ratings_pivot = user_movie_ratings_pivot.withColumn("userId", col("userId").cast("int"))
user_movie_ratings_pivot = user_movie_ratings_pivot.select([col(c).cast("int") for c in user_movie_ratings_pivot.columns])



In [ ]:
# da fare ranking 
# Mappatura movieId -> genres
movie_genres = movies.set_index('movieId')['genres']

# Calcolo della similarità coseno tra le feature latenti dei film
movie_features = Vt.T  # Feature latenti dei film
cosine_sim = cosine_similarity(movie_features)

# Creazione di una mappatura movieId -> indice nella matrice
movie_id_to_index = {movie_id: idx for idx, movie_id in enumerate(user_movie_matrix.columns)}

# Verifica della correlazione tra similarità e generi
similar_movies = {}
for movie_id, genre in movie_genres.items():
    if movie_id in movie_id_to_index:
        idx = movie_id_to_index[movie_id]
        similar_indices = np.argsort(-cosine_sim[idx])[:10]  # Top 10 simili
        similar_movies[movie_id] = [(user_movie_matrix.columns[sim_idx], movie_genres.get(user_movie_matrix.columns[sim_idx], "Unknown")) for sim_idx in similar_indices if user_movie_matrix.columns[sim_idx] in movie_genres.index]

# Output dei risultati
print("Film simili in base alle feature latenti:\n")
for movie_id, similars in list(similar_movies.items())[:5]:  # Mostra i primi 5 risultati
    print(f"Film: {movies.loc[movies['movieId'] == movie_id, 'title'].values[0]} ({movie_genres[movie_id]})")
    for sim_movie_id, sim_genre in similars:
        print(f"  Simile a: {movies.loc[movies['movieId'] == sim_movie_id, 'title'].values[0]} ({sim_genre})")
    print()

In [15]:
spark.stop()